In [33]:
import pandas as pd
import numpy as np
import re
import datetime as dt

In [34]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from seleniumbase import Driver
from urllib import request
from PIL import Image
import pytesseract
from io import BytesIO

import os
from dotenv import load_dotenv

In [35]:
sites = {'인터파크':['https://tickets.interpark.com/contents/search?keyword=','&rows=20&start=0&status=OPENED&status=CLOSED&status=SCHEDULED'],
        '예스이십사':'http://ticket.yes24.com/Search/',
       '멜론':'https://ticket.melon.com/search/index.htm?q=',
       '티켓링크':'https://www.ticketlink.co.kr/search?query='}

In [36]:
concert = pd.read_csv('data/artists_concert.csv', encoding='UTF-8')
concert

,공연명,시작일,종료일,공연시설,공연장,객석수,주최
0,"tripleS FAN-CON, Girls Never Stop",2024.07.06,2024.07.07,이화여자대학교 삼성홀,이화여자대학교 삼성홀,708,(주) 모드하우스
1,"ARTMS World Tour, Moonshot [서울]",2024.07.20,2024.07.21,이화여자대학교 삼성홀,이화여자대학교 삼성홀,708,(주) 모드하우스
2,"DAY6 3RD WORLD TOUR, FOREVER YOUNG [인천]",2024.09.20,2024.09.22,인스파이어리조트,아레나,"15,000",(주)제이와이피엔터테인먼트(JYP Entertainment)
3,온앤오프 (ONF) CONCERT: SPOTLIGHT: THE HIDDEN SHADOW,2024.09.21,2024.09.22,장충체육관,장충체육관,"4,507",WM엔터테인먼트(더블유엠엔터테인먼트)
4,"SUPER JUNIOR, D & E WORLD TOUR: ECLIPSE",2024.09.28,2024.09.29,올림픽공원,SK핸드볼경기장(펜싱경기장),"5,003",오드엔터테인먼트
...,...,...,...,...,...,...,...
101,"Stray Kids, 5-STAR Dome Tour Seoul Special(UNV...",2023.10.21,2023.10.22,고척스카이돔,고척스카이돔,"16,744",(주)제이와이피엔터테인먼트(JYP Entertainment)
102,"VANNER FANCON TOUR, VVS ADVENTURE",2023.10.14,2023.10.15,연세대학교 백주년기념관,콘서트홀,829,주식회사 클렙엔터테인먼트
103,권은비 세 번째 콘서트: QUEEN,2023.10.07,2023.10.08,블루스퀘어,Mastercard Hall(구.아이마켓홀),"1,373",울림엔터테인먼트
104,"NMIXX THE 1ST FAN CONCERT, NMIXX CHANGE UP: MI...",2023.10.07,2023.10.08,장충체육관,장충체육관,"4,507",(주)제이와이피엔터테인먼트(JYP Entertainment)


In [37]:
keywords = concert['공연명'].to_list()
keywords

['tripleS FAN-CON, Girls Never Stop',
 'ARTMS World Tour, Moonshot [서울]',
 'DAY6 3RD WORLD TOUR, FOREVER YOUNG [인천]',
 '온앤오프 (ONF) CONCERT: SPOTLIGHT: THE HIDDEN SHADOW',
 'SUPER JUNIOR, D & E WORLD TOUR: ECLIPSE',
 'Xdinary Heroes Concert, Closed ♭eta: v6.4',
 'IU HEREH WORLD TOUR CONCERT: THE WINNING [서울 (앵콜) ]',
 'CRAVITY FAN CON, BEYOND YOUR MEMORIES',
 'ZEROBASEONE THE FIRST TOUR, TIMELESS WORLD IN SEOUL',
 'ONEWE GOLDEN FESTA: ONEderland',
 'LUCY 1st WORLD TOUR written by FLOWER [서울 (앵콜) ]',
 'Stray Kids World Tour: dominATE [서울]',
 '어썸스테이지, Xdinary Heroes & Dragon Pony & 정수민 [부산]',
 '어썸스테이지, 임현식: BTOB [부산]',
 'BABYMONSTER PRESENTS: SEE YOU THERE [서울]',
 'Xdinary Heroes Concert: Closed beta: v6.3',
 'Red Velvet FANCON TOUR, HAPPINESS: My Dear. ReVe1uv [서울]',
 'Beyond LIVE, THE BOYZ WORLD TOUR: ZENERATION Ⅱ',
 'RIIZE FAN-CON: RIIZING DAY FINALE [서울]',
 'VANNER 1ST CONCERT: THE FLAG: A TO V [서울 (앵콜) ]',
 'DKB WORLD TOUR, DARK STRANGE [서울]',
 'TREASURE: REBOOT, FINAL [서울]',
 '어썸스테이지

In [38]:
keywords_list = [re.split(r':|,', keyword) for keyword in keywords]
keywords_list = [[re.sub(r'\[[^\]]*\]','', s) for s in k] for k in keywords_list]
keywords_list = [[s.strip() for s in k] for k in keywords_list]

keywords_list

[['tripleS FAN-CON', 'Girls Never Stop'],
 ['ARTMS World Tour', 'Moonshot'],
 ['DAY6 3RD WORLD TOUR', 'FOREVER YOUNG'],
 ['온앤오프 (ONF) CONCERT', 'SPOTLIGHT', 'THE HIDDEN SHADOW'],
 ['SUPER JUNIOR', 'D & E WORLD TOUR', 'ECLIPSE'],
 ['Xdinary Heroes Concert', 'Closed ♭eta', 'v6.4'],
 ['IU HEREH WORLD TOUR CONCERT', 'THE WINNING'],
 ['CRAVITY FAN CON', 'BEYOND YOUR MEMORIES'],
 ['ZEROBASEONE THE FIRST TOUR', 'TIMELESS WORLD IN SEOUL'],
 ['ONEWE GOLDEN FESTA', 'ONEderland'],
 ['LUCY 1st WORLD TOUR written by FLOWER'],
 ['Stray Kids World Tour', 'dominATE'],
 ['어썸스테이지', 'Xdinary Heroes & Dragon Pony & 정수민'],
 ['어썸스테이지', '임현식', 'BTOB'],
 ['BABYMONSTER PRESENTS', 'SEE YOU THERE'],
 ['Xdinary Heroes Concert', 'Closed beta', 'v6.3'],
 ['Red Velvet FANCON TOUR', 'HAPPINESS', 'My Dear. ReVe1uv'],
 ['Beyond LIVE', 'THE BOYZ WORLD TOUR', 'ZENERATION Ⅱ'],
 ['RIIZE FAN-CON', 'RIIZING DAY FINALE'],
 ['VANNER 1ST CONCERT', 'THE FLAG', 'A TO V'],
 ['DKB WORLD TOUR', 'DARK STRANGE'],
 ['TREASURE', 'REBOOT

In [39]:
concert_info = pd.DataFrame({'공연명': [],
                            '좌석' : [],
                            '가격' : [],
                            '양도 불가능' : [],
                            '예매처' : []})

In [40]:
found_list = []

In [41]:
# :이랑 , 기준으로 split
# trim()
# 리스트 맨앞(공연 가수 및 종류)으로 
# 사이트 다 돌면서 검색
# len(검색결과) != 0 이면
# 결과의 공연 기간 확인 -> 23.10 ~ 24.09 내면 가져오기
# 리스트 나머지 글자들 가져온 공연명에 다 들어가 있는지 확인
# 다 들어가있는 거만 click()
# 공연가격 가져오기 - 좌석별 차등이면 df 행 3개로 추가
# 상세정보 img OCR -> “양도|제한|예매 취소” 포함하는지 확인
# 있으면 양도 가능에 X, 없으면 O

In [42]:
caution = ['양도', '제한', '예매 취소', '예매취소', '불법', '부정']

In [43]:
def interpark_crawling():
    for k in range(len(keywords_list)):
        # 검색어
        search_word = keywords_list[k][0] if len(keywords_list[k][0]) > 3 else keywords_list[k][1]
        if search_word in found_list:
            continue
    
        url = ''.join([sites['인터파크'][0], search_word, sites['인터파크'][1]])

        driver.get(url)
        driver.switch_to.window(driver.window_handles[0])

        concert_list = driver.find_elements(By.XPATH, '/html/body/div[1]/div/main/div/div/div[1]/div[2]/a')
        if len(concert_list) == 0:
            continue
            
        found_list.append(search_word)
        for i in range(len(concert_list)):
            period = driver.find_element(By.XPATH, f'/html/body/div[1]/div/main/div/div/div/div[2]/a[{i+1}]/ul/li[3]').text
            date = dt.datetime.strptime(period.split(' ')[0].replace('.', '-'), '%Y-%m-%d')

            if date <= to_date and date >= from_date:
                detail = driver.find_element(By.XPATH, f'/html/body/div[1]/div/main/div/div/div/div[2]/a[{i+1}]')
                detail.click()

                # 예매정보 창
                driver.switch_to.window(driver.window_handles[1])

                try:
                    popup = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div/div[3]/button')
                    popup.click()
                except:
                    print('No PopUp')

                driver.implicitly_wait(5)
                title = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[1]/div/div[1]/h2').text
                print(title)
                
                # 상세정보 이미지
                image_elements = []
                try:
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div//p/strong/img[1]'))
                        )
                    image_elements = driver.find_elements(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div//p/strong/img')

                except:
                    print("No img")

                pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

                text = ''
                is_caution = False

                for image_element in image_elements:
                    image_url = image_element.get_attribute('src')
                    res = request.urlopen(image_url).read()
                    img = Image.open(BytesIO(res))
                    text += pytesseract.image_to_string(img, lang='kor')

                is_caution = any(c in text for c in caution)

                # 오픈 공지  - 가격 정보
                driver.get(f"https://ticket.interpark.com/webzine/paper/TPNoticeList.asp?tid1=in_scroll&tid2=ticketopen&tid3=board_main&tid4=board_main")
                driver.switch_to.frame('iFrmNotice')
                name = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, '/html/body//form/div[@class="search"]/input'))
                        )
                name.send_keys(title)
                name.send_keys(Keys.RETURN)
                
                try:
                    # 검색 후 페이지 로딩 대기
                    while len(driver.find_elements(By.XPATH, '/html/body/div//table/tbody/tr')) > 10:
                        continue
                    info = driver.find_element(By.XPATH, '/html/body/div//table/tbody/tr[1]/td[@class="subject"]/a')
                except:
                    concert_info.loc[len(concert_info)] = [title, np.nan, np.nan, is_caution, '인터파크']
                    
                else:
                    info.click()
                    driver.switch_to.default_content()
                    ticket_info = driver.find_element(By.XPATH, '/html/body//div[@class="introduce"]/div[@class="data"]/p[1]').text
                    ticket = ticket_info.split('\n')
                    price = [s for s in ticket if '티켓가' in s or '티켓가격' in s or '티켓 가격' in s]
                    print(price)
                    # 티켓 가격 내용 O
                    if len(price) != 0:
                        price = re.split('\/|:|, ', price[0])[1:]
                        price = [re.sub(r'\([^\]]*\)', '', s) for s in price]
                        price = [s.strip() for s in price]
                        price = [re.sub(r'[^가-힣a-zA-Z0-9 ]|^[0-9]층|원$','', s) for s in price]
                        price = [s.strip() for s in price]
                        price = [[s[:s.rfind(' ')], s[s.rfind(' ')+1:]] for s in price]
                        for j in range(len(price)):
                            concert_info.loc[len(concert_info)] = [title, price[j][0], price[j][1], is_caution, '인터파크']
                    else:
                        concert_info.loc[len(concert_info)] = [title, np.nan, np.nan, is_caution, '인터파크']

                driver.close()
                driver.switch_to.window(driver.window_handles[0])

In [44]:
def yes24_crawling():
    for k in range(len(keywords_list)):
        # 검색어
        search_word = keywords_list[k][0] if len(keywords_list[k][0]) > 3 else keywords_list[k][1]
        if search_word in found_list:
            continue
        url = ''.join([sites['예스이십사'], search_word])

        driver.get(url)
        driver.switch_to.window(driver.window_handles[0])

        concert_list = driver.find_elements(By.XPATH, '/html/body/div[6]/div[4]/div[1]/div[1]/div')
        if len(concert_list) == 0:
            continue
        
        found_list.append(search_word)
        
        # 콘서트별 정보
        for i in range(len(concert_list)):
            period = driver.find_element(By.XPATH, f'/html/body/div[6]/div[4]/div[1]/div[1]/div[{i+1}]/div[3]').text
            date = dt.datetime.strptime(period.split('~')[0].replace('.', '-'), '%Y-%m-%d')

            if date <= to_date and date >= from_date:
                detail = driver.find_element(By.XPATH, f'/html/body/div[6]/div[4]/div[1]/div[1]/div[{i+1}]/div[2]/p[2]/a')
                detail.click()

                # 예매정보 창
                driver.switch_to.window(driver.window_handles[1])
                driver.implicitly_wait(5)
                title = driver.find_element(By.XPATH, '/html/body/form//div/div/div[@class="rn-02"]/p').text
                print(title)

                # 상세정보 이미지
                image_elements = []
                try:
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, '/html/body/form//div/p[1]/img'))
                        )
                    image_elements = driver.find_elements(By.XPATH, '/html/body/form//div/p/img')

                except:
                    print("No img")        
                    
                pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

                text = ''
                is_caution = False

                for image_element in image_elements:
                    image_url = image_element.get_attribute('src')
                    res = request.urlopen(image_url).read()
                    img = Image.open(BytesIO(res))
                    text += pytesseract.image_to_string(img, lang='kor')

                is_caution = any(c in text for c in caution)

                # 오픈 공지 - 가격 정보
                driver.get(f"http://ticket.yes24.com/New/Notice/NoticeMain.aspx")
                name = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div[1]/div[3]/div/input'))
                        )
                name.send_keys(title)
                name.send_keys(Keys.RETURN)
                
                try:
                    # 검색 후 페이지 로딩 대기
                    while len(driver.find_elements(By.XPATH, '/html/body/div[5]/div[1]/div[4]/div/table/tbody/tr/td[2]/a')) > 10:
                        continue
                    info = driver.find_element(By.XPATH, '/html/body/div[5]/div[1]/div[4]/div/table/tbody/tr[2]/td[2]/a')
                except:
                    concert_info.loc[len(concert_info)] = [title, np.nan, np.nan, is_caution, '예스이십사']
                    
                else:
                    info.click()
                    ticket_info = driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div[4]/div[1]/div').text
                    ticket = ticket_info.split('\n')
                    price = [s for s in ticket if '티켓가' in s or '티켓가격' in s or '티켓 가격' in s]
                    print(price)
                    if len(price) != 0:
                        price = re.sub(r'티켓가격* *: *', '', price[0])
                        price = re.split('\/|,', price)
                        price = [re.sub(r'\([^\]]*\)', '', s) for s in price]
                        price = [s.strip() for s in price]
                        price = [re.sub(r'[^가-힣a-zA-Z0-9 ]|^[0-9]층|원$','', s) for s in price]
                        price = [s.strip() for s in price]
                        price = [[s[:s.rfind(' ')], s[s.rfind(' ')+1:]] for s in price]
                        for j in range(len(price)):
                            concert_info.loc[len(concert_info)] = [title, price[j][0], price[j][1], is_caution, '예스이십사']
                    else:
                        concert_info.loc[len(concert_info)] = [title, np.nan, np.nan, is_caution, '예스이십사']



                driver.close()
                driver.switch_to.window(driver.window_handles[0])

In [45]:
def melon_crawling():
    for k in range(len(keywords_list)):
        # 검색어
        search_word = keywords_list[k][0] if len(keywords_list[k][0]) > 3 else keywords_list[k][1]
        if search_word in found_list:
            continue
        url = ''.join([sites['멜론'], search_word])

        driver.get(url)
        driver.switch_to.window(driver.window_handles[0])

        concert_list = driver.find_elements(By.XPATH, '/html/body/div/div[3]/div/div/div/div/ul/li[1]/div/div/table/tbody/tr')
        if len(concert_list) == 0:
            continue   
        
        found_list.append(search_word)
        
        # 콘서트별 정보
        for i in range(len(concert_list)):
            period = driver.find_element(By.XPATH, f'/html/body/div/div[3]/div/div/div/div/ul/li[1]/div/div/table/tbody/tr[{i+1}]/td[2]').text
            date = dt.datetime.strptime(period.split(' ')[0].replace('.', '-'), '%Y-%m-%d')

            if date <= to_date and date >= from_date:
                detail = driver.find_element(By.XPATH, f'/html/body/div/div[3]/div/div/div/div/ul/li[1]/div/div[1]/table/tbody/tr[{i+1}]/td[1]/div/p/a')
                detail.send_keys(Keys.CONTROL +"\n")

                # 예매정보 창
                driver.switch_to.window(driver.window_handles[1])
                # 팝업창 처리
                try:
                    popup = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, '/html/body/div/div[12]/div[1]/div/div/a'))
                        )
                    popup.click()
                except:
                    print('No PopUp')

                driver.implicitly_wait(5)
                title = driver.find_element(By.XPATH, '/html/body/div//p[@class="tit"]').text
                print(title)

                # 상세정보 이미지
                image_elements = []
                try:
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, '/html/body/div/div[3]//div[@class="box_img_content"]//img[1]'))
                        )
                    image_elements = driver.find_elements(By.XPATH, '/html/body/div/div[3]//div[@class="box_img_content"]//img')

                except:
                    print("No img")
                                       
                    
                pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

                text = ''
                is_caution = False

                for image_element in image_elements:
                    image_url = image_element.get_attribute('src')
                    print(image_url)
                    res = request.urlopen(image_url).read()
                    img = Image.open(BytesIO(res))
                    text += pytesseract.image_to_string(img, lang='kor')

                is_caution = any(c in text for c in caution)
                print('양도 불가능 여부:', is_caution)

                # 예매정보창 - 가격 정보
                tickets = driver.find_elements(By.XPATH, '/html/body/div/div[3]//ul[@class="list_seat"]/li')
                if len(tickets) == 0:
                    concert_info.loc[len(concert_info)] = [title, np.nan, np.nan, is_caution, '멜론']
                else:
                    for t in range(len(tickets)):
                        seat = driver.find_element(By.XPATH, f'/html/body/div/div[3]//ul[@class="list_seat"]/li[{t+1}]/span[@class="seat_name"]').text
                        price = driver.find_element(By.XPATH, f'/html/body/div/div[3]//ul[@class="list_seat"]/li[{t+1}]/span[@class="price"]').text
                        price = re.sub(r',|원| ', '', price)
                        print(price)
                        concert_info.loc[len(concert_info)] = [title, seat, price, is_caution, '멜론']
                    

                driver.close()
                driver.switch_to.window(driver.window_handles[0])

In [46]:
def ticketlink_crawling():
    for k in range(len(keywords_list)):
        # 검색어
        search_word = keywords_list[k][0] if len(keywords_list[k][0]) > 3 else keywords_list[k][1]
        if search_word in found_list:
            continue
        url = ''.join([sites['티켓링크'], search_word])

        driver.get(url)
        driver.switch_to.window(driver.window_handles[0])

        concert_list = driver.find_elements(By.XPATH, '/html/body/div/form/main//ul[@class="result_lst"]/li')
        if len(concert_list) == 0:
            continue   
        
        found_list.append(search_word)
        
        # 콘서트별 정보
        for i in range(len(concert_list)):
            period = driver.find_element(By.XPATH, f'/html/body/div/form/main//ul[@class="result_lst"]/li[{i+1}]/div/a[1]/div/dl/dd[2]').text
            date = dt.datetime.strptime(period.split(' ')[0].replace('.', '-'), '%Y-%m-%d')

            if date <= to_date and date >= from_date:
                detail = driver.find_element(By.XPATH, f'/html/body/div/form/main//ul[@class="result_lst"]/li[{i+1}]/div/a[1]')
                detail.send_keys(Keys.CONTROL +"\n")

                # 예매정보 창
                driver.switch_to.window(driver.window_handles[1])
                # 팝업창 처리
                try:
                    popup = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, '/html/body/div//div[@class="common_modal_footer"]/button'))
                        )
                    popup.click()
                except:
                    print('No PopUp')

                driver.implicitly_wait(5)
                title = driver.find_element(By.XPATH, '/html/body/div/div/main//h1[@class="product_title"]').text
                print(title)

                # 상세정보 이미지
                image_elements = []
                try:
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, '/html/body/div/div/main//section/div[@class="product_templete"]/p/img[1]'))
                        )
                    image_elements = driver.find_elements(By.XPATH, '/html/body/div/div/main//section/div[@class="product_templete"]/p/img')

                except:
                    print("No img")
                                       
                    
                pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

                text = ''
                is_caution = False

                for image_element in image_elements:
                    image_url = image_element.get_attribute('src')
                    print(image_url)
                    res = request.urlopen(image_url).read()
                    img = Image.open(BytesIO(res))
                    text += pytesseract.image_to_string(img, lang='kor')

                is_caution = any(c in text for c in caution)
                print('양도 불가능 여부:', is_caution)

                # 예매정보창 - 가격 정보
                tickets = driver.find_elements(By.XPATH, '/html/body/div/div/main//ul/li[@class="product_info_subitem"]')
                if len(tickets) == 0:
                    concert_info.loc[len(concert_info)] = [title, np.nan, np.nan, is_caution, '티켓링크']
                else:
                    for t in range(len(tickets)):
                        ticket = tickets[t].text.split(' ')
                        seat = ticket[0]
                        price = ticket[1]
                        price = re.sub(r',|원| ', '', price)
                        print(price)
                        concert_info.loc[len(concert_info)] = [title, seat, price, is_caution, '티켓링크']
                    

                driver.close()
                driver.switch_to.window(driver.window_handles[0])

In [47]:
concert_info

,공연명,좌석,가격,양도 불가능,예매처


In [48]:
driver = Driver(browser='chrome', uc=True)

from_date = dt.datetime.strptime('2023-10-01', '%Y-%m-%d')
to_date = dt.datetime.strptime('2024-09-30', '%Y-%m-%d')

# 크롤링 준비
driver.implicitly_wait(10)
driver.set_window_size(1100, 1100)


# 사이트별 크롤링
interpark_crawling()
yes24_crawling()
melon_crawling()
ticketlink_crawling()

No PopUp
2024 온앤오프(ONF) CONCERT ［SPOTLIGHT］ IN SEOUL
['- 티켓 가격: 선예매 - 132,000원/ 일반예매 - 143,000원']
No PopUp
2024 온앤오프(ONF) CONCERT ［SPOTLIGHT:THE HIDDEN SHADOW］
['▷ 티켓가: 전석: 154,000원 / 시야제한석: 132,000원']
2024 ZEROBASEONE THE FIRST TOUR ［TIMELESS WORLD］ IN SEOUL
['▷ 티켓가격: VIP석 198,000원, 일반석 154,000원 (부가세 포함)']
No PopUp
LUCY 1st WORLD TOUR written by FLOWER in SEOUL
['- 티켓가격 : 전석 132,000원']
No PopUp
LUCY 1st WORLD TOUR written by FLOWER ENCORE in SEOUL
['- 티켓가격 : 전석 132,000원']
No PopUp
2024 어썸스테이지 ［ 서은광 x 임현식 ］ - 부산
No PopUp
2024 어썸스테이지 ［ N.Flying ］
['- 티켓가격 : R석 121,000원 / S석 110,000원 / A석 99,000원']
No PopUp
2024 어썸스테이지 ［ 로이킴 x 멜로망스 ］ - 부산
No PopUp
2024 어썸스테이지 ［ DAY6 ＆ 오월오일 ］- 부산
['- 티켓가격 : R석 132,000원 / S석 121,000원 / A석 110,000원 / 시야제한석 110,000원 / 시야방해석 99,000원']
No PopUp
2024 어썸스테이지 ［ 이민혁 (HUTA) ］ - 부산
No PopUp
2024 어썸스테이지 ［ DAY6 ＆ Xdinary Heroes ］- 부산
['- 티켓가격 : R석 132,000원 / S석 121,000원 / A석 110,000원 / 시야제한석 110,000원 / 시야방해석 99,000원']
No PopUp
2024 어썸스테이지 ［ 적재 x 김재환 ］ - 부산
No PopUp
20

https://cdnticket.melon.co.kr/resource/image/upload/product/2024/01/2024012315395614fba9f8-3270-4876-8a1e-72243ca493c8.jpg/melon/quality/90/optimize
양도 불가능 여부: True
165000
165000
165000
165000
154000
154000
154000
154000
132000
132000
132000
132000
110000
110000
110000
110000
WOODZ World Tour ‘OO－LI’ FINALE
https://cdnticket.melon.co.kr/resource/image/upload/product/2024/01/2024010318041293425b66-16f0-4627-9c16-9b3af7192224.jpg/melon/quality/90/optimize
양도 불가능 여부: True
132000
132000
121000
110000
2023 WOODZ World Tour ‘OO－LI and’ in Seoul
https://cdnticket.melon.co.kr/resource/image/upload/product/2023/10/202310050904321b19faf3-c6ae-4eeb-adef-bf93b4e9332c.jpg/melon/quality/90/optimize
양도 불가능 여부: True
132000
132000
121000
No PopUp
퍼플키스（PURPLE KISS）FAN CONCERT 〈FESTA〉
https://cdnticket.melon.co.kr/resource/image/upload/product/2023/12/202312191332468d703148-6155-4539-9091-f772981eb79a.png/melon/quality/90/optimize
양도 불가능 여부: True
77000
2023 RIIZE 100 DAYS PARTY ‘Hello，BRIIZE’
https://cdn

In [78]:
concert_info[0:60]

,공연명,좌석,가격,양도 불가능,예매처
0,2024 온앤오프(ONF) CONCERT ［SPOTLIGHT］ IN SEOUL,선예매,132000,1.0,인터파크
1,2024 온앤오프(ONF) CONCERT ［SPOTLIGHT］ IN SEOUL,일반예매,143000,1.0,인터파크
2,2024 온앤오프(ONF) CONCERT ［SPOTLIGHT:THE HIDDEN S...,전석,154000,1.0,인터파크
3,2024 온앤오프(ONF) CONCERT ［SPOTLIGHT:THE HIDDEN S...,시야제한석,132000,1.0,인터파크
6,2024 ZEROBASEONE THE FIRST TOUR ［TIMELESS WORL...,VIP석,198000,1.0,인터파크
7,2024 ZEROBASEONE THE FIRST TOUR ［TIMELESS WORL...,일반석,154000,1.0,인터파크
8,LUCY 1st WORLD TOUR written by FLOWER in SEOUL,전석,132000,1.0,인터파크
9,LUCY 1st WORLD TOUR written by FLOWER ENCORE i...,전석,132000,1.0,인터파크
10,2024 어썸스테이지 ［ 서은광 x 임현식 ］ - 부산,NaN,NaN,1.0,인터파크
11,2024 어썸스테이지 ［ N.Flying ］,R석,121000,1.0,인터파크


In [61]:
concert_info.loc[2, ['좌석', '가격']] = ['전석', 154000]
concert_info.loc[3, ['좌석', '가격']] = ['시야제한석', 132000]
concert_info.loc[61, ['좌석', '가격']] = ['전석', 132000]
concert_info.loc[85, ['좌석', '가격']] = ['전석', 110000]
concert_info.loc[98, ['좌석', '가격']] = ['전석', 154000]
concert_info.loc[[100, 102, 104, 106, 108, 110], ['좌석', '가격']] = ['전석', 132000]
concert_info.loc[117, ['좌석', '가격']] = ['전석', 154000]
concert_info.loc[120, ['좌석', '가격']] = ['전석', 143000]
concert_info.loc[122, ['좌석', '가격']] = ['전석', 165000]
concert_info.loc[123, ['좌석', '가격']] = ['전석', 121000]
concert_info.loc[127, ['좌석', '가격']] = ['전석', 132000]
concert_info.loc[129, ['좌석', '가격']] = ['전석', 143000]
concert_info.loc[132, ['좌석', '가격']] = ['전석', 154000]
concert_info.loc[137, ['좌석', '가격']] = ['일반예매', 99000]
concert_info.loc[139, ['좌석', '가격']] = ['팬클럽 선예매', 88000]
concert_info.loc[142, ['좌석', '가격']] = ['전석', 143000]
concert_info.loc[145, ['좌석', '가격']] = ['전석', 132000]
concert_info.loc[147, ['좌석', '가격']] = ['팬클럽 선예매', 99000]
concert_info.loc[145, ['좌석', '가격']] = ['일반예매', 11000]
concert_info.loc[203, ['좌석', '가격']] = ['전석', 132000]

In [64]:
concert_info = concert_info.drop([4, 5, 27, 28, 29, 30, 36, 43, 44, 46, 47, 49, 50, 52, 53, 62, 70, 71, 72, 73, 74, 75, 77,
                                  78, 79, 80, 81, 82, 83, 84, 86, 94, 99, 101, 103, 105, 107, 109, 111, 115, 116, 118, 119, 121,
                                  124, 125, 128, 130, 133, 138, 140, 141, 143, 146, 148, 150, 207, 208])

In [76]:
concert_info

,공연명,좌석,가격,양도 불가능,예매처
0,2024 온앤오프(ONF) CONCERT ［SPOTLIGHT］ IN SEOUL,선예매,132000,1.0,인터파크
1,2024 온앤오프(ONF) CONCERT ［SPOTLIGHT］ IN SEOUL,일반예매,143000,1.0,인터파크
2,2024 온앤오프(ONF) CONCERT ［SPOTLIGHT:THE HIDDEN S...,전석,154000,1.0,인터파크
3,2024 온앤오프(ONF) CONCERT ［SPOTLIGHT:THE HIDDEN S...,시야제한석,132000,1.0,인터파크
6,2024 ZEROBASEONE THE FIRST TOUR ［TIMELESS WORL...,VIP석,198000,1.0,인터파크
...,...,...,...,...,...
202,2024 tripleS FAN-CON < Girls Never Stop >,전석,110000,1.0,티켓링크
203,2024 ARTMS World Tour < Moonshot > in Seoul,전석,132000,1.0,티켓링크
204,뮤직플래닛 2024_ LUCY X N.Flying,R석,121000,1.0,티켓링크
205,뮤직플래닛 2024_ LUCY X N.Flying,S석,110000,1.0,티켓링크


In [77]:
concert_info.to_csv("data/ticket_site_concert_info.csv", index=False)